# __MONKEYPOX PREDICTION__

# 1. BUSINESS UNDERSTANDING

## Overview

Monkeypox, according to the Centers for Disease Control and Prevention, is an uncommon disease caused by infection with the monkeypox virus. Monkeypox virus is part of the same family of viruses as variola virus, the virus that causes smallpox. Monkeypox symptoms are similar to smallpox symptoms, but milder, and monkeypox is rarely fatal.
Monkeypox was discovered in 1958 when two outbreaks of a pox-like disease occurred in colonies of monkeys kept for research. Despite being named “monkeypox,” the source of the disease remains unknown. However, African rodents and non-human primates (like monkeys) might harbor the virus and infect people.

The first human case of monkeypox was recorded in 1970. Prior to the 2022 outbreak in May, monkeypox had been reported in people in several central and western African countries. Previously, almost all monkeypox cases in people outside of Africa were linked to international travel to countries where the disease commonly occurs or through imported animals. These cases occurred on multiple continents.

```Transmission```: Monkeypox is a viral zoonosis (virus transmitted to humans from animals)It is transmitted through close contact with an infected person or animal, body fluids, respiratory droplets and contaminated materials such as bedding.

## Problem Statement

As more cases of monkeypox emerge, the world becomes increasingly concerned, as they do not want a repeat of what happened not too long ago, when the infamous Covid-19 broke out and altered the world as we know it. Faced with the reality that such viral diseases can actually shut down the planet, the organization must develop ways to reduce the spread of this disease, enter the Health authorities in the United Kingdom, they have established an incident management team to coordinate the extensive contact tracing  in regards to how quickly the virus can be transferred and, worse, mutate into a more lethal variant.

On July 23, the World Health Organization declared the outbreak “a public health emergency of international concern.” Another thing you should know about monkeypox is that it is extremely unpleasant. The current strain has a 1% mortality rate, and as of this writing, there have been three deaths reported in outbreaks outside of Africa, and five deaths in African countries where the disease is endemic, since the beginning of the year.
The West African strain of the monkeypox virus is associated with milder disease and fewer deaths than the Central African strain.

Even if the chances of death are low, patients with monkeypox have reported how painful and debilitating the disease is. "People may experience flu-like symptoms at first, such as fever and headache, but as the disease progresses, you get a multi-stage rash, lesions can develop in your mouth, feet, and genital region, and these develop into pus-filled blisters."Symptoms can appear between five and 21 days after infection, though the average incubation period is 6 to 13 days. For the first few days, there are headaches, fever, muscle aches, and fatigue.

As students of the Incident Management team we have been tasked to create a model to accurately identify potential infected persons in order to make contact tracing faster and more efficient, thereby halting the spread.

## Proposed Solution

One proposed solution is to cultivate the use of data in attempting to predict whether or not a person has monkeypox in order to make contact tracing easier.

## Specific Objectives

* To predict whether a patient is negative or positive for monkeypox based on the symptoms they exhibit 
* To analyze the various variables such as Sore Throat,Penile Oedema, Oral Lesions ,Systemic illness and STIs and know their relationship with monkeypox 

## Research Questions

* Which model best predicts monkeypox disease?
* Which symptom has the highest correlation to monkeypox?
* Which symptom has the lowest correlation to monkeypox?

## Success Criteria

Tentatively, the study will be judged a success if, we build a model to predict monkeypox with a recall of 70% or more in classification



## Importing Libraries

Libraries to enhance data manipulation.

These comprises of;
   > **NumPy**
   
   NumPy is an open source project aiming to enable numerical computing with Python.
   
   > **pandas**
   
   pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
    built on top of the Python programming language.   

In [1]:
import pandas as pd
import numpy as np

Libraries for visualization;

   >**Seaborn**
    
   Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics.
    
   >**Matplotlib**
    
   Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python.

In [2]:
import seaborn as sns
# setting the theme
sns.set_style("darkgrid")

import matplotlib.pyplot as plt

Libraries for modelling tasks;

   >**scikit-learn**
   
   Simple and efficient tools for predictive data analysis, accessible to everybody, and reusable in various contexts built on NumPy, SciPy, and matplotlib

In [ ]:
!pip install imbalanced-learn

In [ ]:
from sklearn.linear_model import Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import recall_score, precision_score ,accuracy_score, plot_confusion_matrix, confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE, ADASYN 

Library for saving our final model

In [ ]:
import joblib

Library for ignoring deprecation warnings

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

#### Defining relevant Functions.

In [ ]:
# function to display the values on the graph
def show_values(axs, orient="v", space=.01):
    def _single(ax):
        if orient == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height() + (p.get_height()*0.01)
                value = '{:.1f}'.format(p.get_height())
                ax.text(_x, _y, value, ha="center") 
        elif orient == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height() - (p.get_height()*0.5)
                value = '{:.1f}'.format(p.get_width())
                ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _single(ax)
    else:
        _single(axs)
        
# function to plot the pie charts
def univariate(name):
    """ Function to plot the pie charts """

    figure = plt.pie(mp[name].value_counts(), labels = mp[name].unique(),
             startangle = 90, shadow = True, autopct = "%2.1f%%")
    plt.title(name)
    return figure

#function to train model and return the score
def model_trainer(model, X_tr, y_tr, X_te, y_te):
    model.fit(X_tr, y_tr)
    pre = model.predict(X_te)
    precision =precision_score(y_te, pre)
    recall =recall_score(y_te, pre)
    accuracy =accuracy_score(y_te, pre)
    f1 = f1_score(y_te, pre)
    
    return {'precision':precision,
            'recall_score':recall,
            'accuracy_score':accuracy,
            'f1_score':f1}

#function to tune the model and find best parameters for the DecisionTreeClassifier using gridSearchCv
def grid_search(model, parms, X_tr, y_tr):
    gs_tree = GridSearchCV(model, parms, cv=3)
    gs_tree.fit(X_tr, y_tr)

    return gs_tree.best_params_

## Loading Data

In [ ]:
# Loading the data using pandas into mp {Monkey Pox}

# Index_col sets the index to be the patient id as there is no need to have more than one unique identifiers.

mp = pd.read_csv('DATA.csv')

# 2. DATA UNDERSTANDING

## Overview

We are using a SYNTHETIC dataset generated from a study published by the British Medical Journal. It is a description of the clinical features and novel presentations of human monkeypox during the outbreak of 2022 in central London. 

The data has already been loaded in the above code cell.

In [ ]:
# preview of the first 5 rows
mp.head()

In [ ]:
# Checking the summary of the data

mp.info()

In [ ]:
# checking for duplicates using the index as a guide as the index is the personal id
mp.duplicated().sum()

In [ ]:
# viewing the column names of the dataset 

mp.columns

In [ ]:
# checking the summary statistics of the data
mp.describe()

In [ ]:
# shape of the data
mp.shape

In [ ]:
# Checking on the uniue value counts in Systemic Illness
mp['Systemic Illness'].value_counts()

In [ ]:
# Reducing ambiguity in the systemic illness column we change none to No systemic Illness

mp['Systemic Illness'].replace(to_replace= 'None',value= 'No_Systemic_Illness',inplace= True)

In [ ]:
# confirming
mp['Systemic Illness'].value_counts()

__Summary__

* Our data has 25,000 rows and 11 columns
* The data has 8 columns of dtype ```Bool``` and 2 columns of dtype ```object```.
* There are no missing columns in the data.
* There are no duplicates in the data.
* One column in particular will be dropped `Patient_ID` as the patient id wont be helpful in the modeling as it will only add noise for the machine.
* Uniformity of column names will also be done to reduce ambiguity.
* The following are the relevant columns in our data;
    * Systemic Illness 
    * Rectal Pain 
    * Sore Throat 
    * Penile Oedema
    * Oral Lesions 
    * Solitary Lesion 
    * Swollen Tonsils 
    * HIV Infection
    * Sexually Transmitted Infection
    * MonkeyPox
    


Systemic illness has the most value counts after Patient_ID with 4.

data cleaning will be relatievly easy as the data has no missing values and no duplicates.


Since the data has no duplicates and missing values, we can go ahead and perfom column name uniforminty, type casting. and drop the `Patient_ID` before starting on Exploratory Data Analysis to gain more insights from the data.

### Dropping Patient ID

In [ ]:
# dropping the patient id
mp.drop(columns=['Patient_ID'],axis = 1, inplace=True)
mp.head(3)

### Uniformity of Column names

In [ ]:
# replacing the names
mp.rename(columns={'Systemic Illness':'Systemic_Illness',
                        'Rectal Pain':'Rectal_Pain',
                        'Sore Throat':'Sore_Throat',
                        'Penile Oedema':'Penile_Oedema',
                        'Oral Lesions':'Oral_Lesions',
                        'Solitary Lesion':'Solitary_Leision',
                        'Swollen Tonsils':'Swollen_Tonsils',
                        'HIV Infection':'HIV_Infection',
                        'Sexually Transmitted Infection':'STI',
                        'MonkeyPox':'Target'},inplace=True)
mp.head(3)

### Type casting Objects to Categorical data types

In [ ]:
# For loop to get all object types in the data frame
col_obj = []
for x in list(mp.columns):
    if mp[x].dtypes == object:
        col_obj.append(x)
col_obj        

In [ ]:
# type_casting object to categorical 

mp[col_obj] = mp[col_obj].astype('category')

# confirming
mp.info()

## Exploratory Data Analysis

### UNIVARIATE ANALYSIS

__Feature columns__

In univariate analysis, we will investigate the distribution of symptoms using pie charts.

In [ ]:
# Making a list of column names to use in the notebook
col = list(mp.columns)

In [ ]:
# columns to use when plotting the pie charts
columns =  col[:-1]

# plotting the pie charts
plt.figure(figsize = (15, 20))
for i in enumerate(columns):
    plt.subplot(3, 3, i[0] + 1)
    univariate(i[1])

__Target Column__

In [ ]:
univariate("Target");

__Univariate Analysis summary__

* From the above, all symptoms have an almost uniform distribution based on whether it is true or false.
* For the target column, 63.6% of the data consists of those who are monkeypox negative and 36.4% are those who are monkeypox positive therefore there is a class imbalance in our target column.

### BIVARIATE ANALYSIS

In Bivariate analysis, we will compare the feature columns to our target variable to see the relationship between the two.
This will be achieved by plotting side by side bar charts to see the relationship.

In [ ]:
# Plotting side by side graphs for the bivariate analysis

features = col[:-1]
plt.figure(figsize = (18, 38))
for idx,val in enumerate(features):
    plt.subplot(5, 2, idx + 1)
    sns.countplot(x = val, hue = "Target", data = mp )
    plt.xticks(rotation = 10)
    plt.title(val.upper())   

__Bivariate Analysis Summary__

* For the systemic illness symptoms, people with ```fever``` and ```swollen lymph nodes``` have a high chance of being MonkeyPox positive.
* People with the symptoms of ```HIV Infection```, ```Rectal Pain``` and ```Sexually Transmitted Infection``` also have a high chance of being MonkeyPox positive.
* They all 

### INVESTIGATING CORRELATION

In [ ]:
plt.figure(figsize = (20, 10))
var = mp.corr()
sns.heatmap(var, annot = True, cmap = "YlGnBu");

No high correlation was observed in the dataset.

# 3. DATA PREPARATION

Now that we have finished with EDa. We should prepare the data `mp` for modeling by one hot encoding to get binary column.


In [ ]:
col

In [ ]:
# type casting Systemic illness by using get dummies
df = pd.get_dummies(data= mp, columns= ['Systemic_Illness'],drop_first= False)
# dropping every other column to isolate the systemic illness ...will concat later.
df_Systemic = df.drop(columns= col[1:],axis = 1)
# df_Systemic = df_Systemic.drop(columns= ['Systemic Illness_No Systemic Illness'],axis = 1)
df_Systemic

In [ ]:
# drop_first is set to true to reduce on multicoliniearity
df_d = pd.get_dummies(data = mp, columns= col[1:], drop_first= True)
df_d

In [ ]:
# Dropping Systemic illness as its already dummied out

df_d.drop(axis = 1, columns= ['Systemic_Illness'] ,inplace = True)


In [ ]:
# Column names will have to be changed but that can be done in one go
# first we finish up on converting the column data types t a usable state.
df_d.head()

In [ ]:
df_clean = pd.concat([df_Systemic,df_d], axis = 1,join= 'inner')
df_clean

In [ ]:
# Rechecking the multicolinearity to ensure that data preparation has not introduced any
plt.figure(figsize = (20, 10))
var = df_clean.corr()
sns.heatmap(var, annot = True, cmap = "YlGnBu");

In [ ]:
list(df_clean.columns)

In [ ]:
df_clean.rename(columns={'Systemic_Illness_Fever':'SI_Fever',
                        'Systemic_Illness_Muscle Aches and Pain':'SI_Muscle_Aches&Pain',
                        'Systemic_Illness_Swollen Lymph Nodes':'SI_Swollen_Lymph_Nodes',
                        'Rectal_Pain_True':'Rectal_Pain',
                        'Sore_Throat_True':'Sore_Throat',
                        'Penile_Oedema_True':'Penile_Oedema',
                        'Oral_Lesions_True':'Oral_Lesions',
                        'Solitary_Lesion_True':'Solitary_Leision',
                        'Swollen_Tonsils_True':'Swollen_Tonsils',
                        'HIV_Infection_True':'HIV_Infection',
                        'STI_True':'STI',
                        'Target_Positive':'Target'},inplace=True)

In [ ]:
df_clean.head()

In [ ]:
# splitting the data into X [predictors] and y [target]
X = df_clean.drop(columns= ['Target'],axis = 1)
y = df_clean['Target']

# Random state for reapetability, test size of 30% as it is the optimum
# Stratify is used to ensure the test and train data have the same ratio

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size= 0.3,stratify= y)

# 4. MODELLING

In [ ]:
#checking class balance in the training target variable
print(y_train.value_counts())
print('\n')

#checking class balance in the training target variable
print(y_test.value_counts())

An imbalance between positive and negative is observed

# Base Model DecisionTreeClassifier

In [ ]:
#train and get the score of the base model without handling the imbalance in the class and see how the model performs
base_tree =  DecisionTreeClassifier()
model_trainer(base_tree, X_train, y_train,X_test,y_test)

A recall of 82 % and precision of 71% is a good start now lets tune it to look for improvements. 

In [ ]:
#visualize the confusion matrix
cnf_matrix  = plot_confusion_matrix(base_tree, X_test, y_test, cmap=plt.cm.Blues)

In [ ]:
#balance the training classes and see the effect on the model using SMOTE
X_train_smote, y_train_smote = SMOTE().fit_resample(X_train, y_train)

print(y_train_smote.value_counts())

In [ ]:
#retrain the model with the model with the balanced classes
base_tree_smote = DecisionTreeClassifier()

Basemodel_balanced = model_trainer(base_tree_smote, X_train_smote, y_train_smote,X_test,y_test)

Basemodel_balanced

Precision has increased to 74% which is good, but the recal has reduced to 63%.

In [ ]:
#visualize the confusion matrix
cnf_matrix  = plot_confusion_matrix(base_tree_smote, X_test, y_test, cmap=plt.cm.Blues)

In [ ]:
#balanced class best perameters
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [1, 5, 10, 20],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6]
  }
grid_search(DecisionTreeClassifier(),param_grid, X_train_smote, y_train_smote)


In [ ]:
#  Using the best parameters.
dtree_par =  DecisionTreeClassifier(criterion= 'gini', max_depth = 15, min_samples_split= 5,min_samples_leaf = 1)
DecisionTree = model_trainer(dtree_par, X_train_smote,y_train_smote, X_test,y_test)
DecisionTree

In [ ]:
cnf_matrix  = plot_confusion_matrix(dtree_par, X_test, y_test, cmap=plt.cm.Blues)

The model recall score increased to 64% meaning more positive patients were identified but there are still many wrongly classified patients

Whereas the precision score maintained at 74% , dropped by 0.003 when decimal places are considered.


## LogisticRegression and KNeighborsClassifier

In [ ]:
#evaluating the two more models and see if they perform better then the DecisionTreeClassifier
# these two are LogisticRegression() and KNeighborsClassifier()

logreg = LogisticRegression()
knn = KNeighborsClassifier()

models = [logreg,knn]
scores=[]
recall=[]
scores_2 = []
precision_=[]

for i in models:
    estimator=Pipeline([
        ('model',i)])
    Prec_ =cross_val_score(estimator,X_train_smote,y_train_smote,cv=3,scoring='precision')
    recall_scre=cross_val_score(estimator,X_train_smote,y_train_smote,cv=3,scoring='recall')
    scores.append(recall_scre)
    scores_2.append(Prec_)
    recall.append(recall_scre.mean())
    precision_.append(Prec_.mean())
    
    
pd.DataFrame({'model':['Logistic Regression','KNN'],'Precision score':precision_,'Recall Score':recall}).set_index('model').sort_values(by='Recall Score',ascending=False)

recall score shoots up to 82% but Precision score drops to 58%

### KNeighbors

In [ ]:
#plot the KNN and virtualize the confussion matrix

kn_base =  KNeighborsClassifier()
model_trainer(kn_base, X_train_smote, y_train_smote,X_test,y_test)

precision score at 70% while recall score at 77%

In [ ]:
#tune the the KNN 
knn_gridSearch = KNeighborsClassifier()

parameters = {'n_neighbors':[10,15,22,30],
              'leaf_size':[3,4,5,8,10],
              'algorithm':['auto', 'kd_tree'],
              'n_jobs':[-1]}
grid_search(knn_gridSearch, parameters, X_train_smote, y_train_smote)


In [ ]:
# Using the best parameters 
knn_improved = KNeighborsClassifier(algorithm = 'auto', leaf_size = 3, n_jobs = -1, n_neighbors = 22)
KNeighbors = model_trainer(knn_improved, X_train_smote,y_train_smote, X_test,y_test)
KNeighbors

Precision raises to 73% while recall drops to 71%

In [ ]:
cnf_matrix  = plot_confusion_matrix(knn_improved, X_test, y_test, cmap=plt.cm.Blues)

The KNN Model has improved our recall significantly and this means our model will be able to identify more people with the  virus

### Logistic Regression

In [ ]:
#final model the logisticRegressor
base_logrig = LogisticRegression()
model_trainer(base_logrig, X_train_smote, y_train_smote,X_test,y_test)

precision score is 74 % while recall is at an unacceptable low of 59%

In [ ]:
#performing hyper perameter tuning on the model
logR_improved = LogisticRegression()

parameters = { 'C': np.logspace(-1, 1, 50),
                'penalty' : ['l1', 'l2'],
             'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
             'max_iter': [50,100,150,200]}
grid_search(logR_improved, parameters, X_train_smote, y_train_smote)


In [ ]:
# Using the best parameters
logR = LogisticRegression(C = 0.00030888435964774815, penalty ='l2',solver = 'liblinear')
LogisticReg = model_trainer(logR, X_train_smote, y_train_smote,X_test,y_test)
LogisticReg

In [ ]:
cnf_matrix  = plot_confusion_matrix(logR, X_test, y_test, cmap=plt.cm.Blues)

Precision score and recall score have not changed

### Random Forest Classifier

In [ ]:
pipe = Pipeline([
                    ('Rfc',RandomForestClassifier(random_state=0,))
                               
                ])
min_range = list(np.arange(0.08,0.14,0.02))
grid_params = [{
               'Rfc__n_estimators':[50,100],
               'Rfc__min_samples_leaf':min_range,
               'Rfc__max_features':['sqrt','log2'],
               'Rfc__max_leaf_nodes':[12,13],
               'Rfc__bootstrap':[True,False]
               }]

gs = GridSearchCV(estimator= pipe, param_grid= grid_params, cv= 10,scoring='precision')

gs.fit(X_train,y_train)

# Best accuracy
print('Best precision: %.3f' % gs.best_score_)
print('_________')
# Best params
print('\nBest params:\n', gs.best_params_)
print('_________')

y_pred = gs.predict(X_test)

print('_________')

print(precision_score(y_true= y_test,y_pred= y_pred))

In [ ]:
# Using the best parameters
RandomForest = RandomForestClassifier(bootstrap = False, max_features = 'sqrt',max_leaf_nodes = 12,
                                      min_samples_leaf= 0.08,n_estimators= 50)
RandomForestClass = model_trainer(RandomForest, X_train_smote, y_train_smote,X_test,y_test)
RandomForestClass

In [ ]:
scores = [LogisticReg, Basemodel_balanced, DecisionTree,KNeighbors,RandomForestClass ]
column_name = ['LogisticReg','Basemodel', 'DecisionTree','KNeighbors','RandomForestClassifier' ]

In [ ]:
df = pd.DataFrame(columns = column_name)
df.LogisticReg =LogisticReg.values()
df.Basemodel = Basemodel_balanced.values()
df.DecisionTree = DecisionTree.values()
df.KNeighbors = KNeighbors.values()
df.RandomForestClassifier = RandomForestClass.values()
df.index = ['precision', 'recall_score','accuracy_score','f1_score']
df

The KNeighborsClassifier had the best sores and since this model will be used in detecting MonkeyPox viruses a high recall is needed then precision since this will prevent the number of false positives making more positive cases to be identified and treated in advance

In [ ]:
#develop a final pipline that will be used to train the model on the whole dataset and make the model ready for deployment
#Balance the whole dataset using smote 
X_smote, y_smote = SMOTE().fit_resample(X, y)
y_smote.value_counts()

In [ ]:
pipeline = Pipeline([('KNN', KNeighborsClassifier(algorithm = 'auto', leaf_size = 4, n_jobs = -1, n_neighbors = 8))])
pipeline.fit(X_smote,y_smote)

In [ ]:
#model performance after training on the whole data
model_trainer(pipeline, X_smote, y_smote,X_test,y_test)


In [ ]:
#save our model into a pickle
with open('model.pkl', 'wb') as f:
  joblib.dump(pipeline, f)

# 5. EVALUATION

In [ ]:
df = pd.DataFrame(columns = column_name)
df.LogisticReg =LogisticReg.values()
df.Basemodel = Basemodel_balanced.values()
df.DecisionTree = DecisionTree.values()
df.KNeighbors = KNeighbors.values()

df.index = ['precision', 'recall_score','accuracy_score','f1_score']
df

The differences in accuracy are low thus other evaluation metrics are required. In this case, we shall use recall to evaluate the various models and decide on the best one. Based on the recall scores, The KNeighbors model displays both high precision meaning it produces correct positive predictions about 71.74% percent of the time. It also displays a comparatively high recall score which means it does not produce a lot of false negatives. 

In [ ]:
log_prediction =base_logrig.predict(X_test)
knn_prediction =knn_improved.predict(X_test)

In [ ]:
#evaluating the scores for prediction
from sklearn.metrics import classification_report

print ("CLassification report - Logistic Regression\n", 
       classification_report(y_test, log_prediction, target_names = [ "FRAUD","VALID"]))
print ("CLassification report - KNN Classifier\n", 
       classification_report(y_test, knn_prediction, target_names = ["FRAUD","VALID"]))

In [ ]:
#previously defined scores for the training for comparison with the previous cell
KNeighbors

Using the classification report library we can see that our model performs equally as well in the prediction. This means that it was well fitted.The consistency in precision and recall in the KNN model merited it as the better model over the logistic regression. 

# 6. DEPLOYMENT